# imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# functions and variables

## variables

In [2]:
target_action = ['sub_car_claim_click', 'sub_car_claim_submit_click',
'sub_open_dialog_click', 'sub_custom_question_submit_click',
'sub_call_number_click', 'sub_callback_submit_click', 'sub_submit_success',
'sub_car_request_submit_click']
drop_columns_hit = ['event_label', 'event_value', 'hit_referer', 'hit_type']
drop_columns_sessions = ['device_model', 'utm_keyword']
organic_traffic = ['organic', 'referral', '(none)']

## functions

In [3]:
def unique_values_column(df):
    for column in df.columns:
        print(f'column {column} unique: {len(df[column].unique())}')

In [4]:
def categorical_values(df):
    category_value = pd.Series()
    for column in df.columns:
        category_value.loc[column] = len(df[column].unique())
    print(f'all line: {df.shape[0]}')
    return category_value

In [5]:
def percent_null(df):
    print(f'values line: {df.shape[0]}, columns: {df.shape[1]}')
    sum_null = df.isna().sum()
    sum_line = df.shape[0]
    equally = sum_null / sum_line *100
    print('Процент отсутствующих данных в df:')
    return round((equally), 2)

In [6]:
###
##
#---> Функция ищет медиану по извесной информации о проведённом времени 
#---> в аналогичных категориях и заменяет ей nan значения в hit_time
##
###
def hit_time_median_category_event(df: pd.DataFrame) -> pd.DataFrame: 
    df_finish = pd.DataFrame()
    median_time = 0
    #category_median = {}
    
    df_temp = df[['hit_time', 'event_category']].copy()
    categories = df_temp['event_category'].value_counts().index.tolist()
    
    for category in categories:
        median_time = df_temp[df_temp['event_category'] == category].hit_time.quantile(0.50)
        #category_median[category] = median_time
        
        df_temp1 = df_temp[((df_temp.hit_time.isna() == True)
                         & (df_temp['event_category'] == category))].hit_time.fillna(median_time)
        df_finish = pd.concat([df_finish, df_temp1])

    df_finish = df_finish.rename(columns={0: 'hit_time'})    
    df.loc[df_finish.index, 'hit_time'] = df_finish['hit_time']
        
    return df#, category_median

In [7]:
###
##
#---> Функция создает таргет 1 для ключевых слов и 0 для оставшихся. Сортирует
#---> столбик с таргетами чтобы все таргеты = 1 были первыми и удаляет все
#---> дубли поле первого уникального значения сессии. Возвращает DF сохряняя индексы.
##
###
def no_duplicated_target_and_null(df: pd.DataFrame, 
                                  target_action = target_action, 
                                  new_column, 
                                  column_target, 
                                  drop_dup_column) -> pd.DataFrame:
    
    df_temp = df.copy()
    pattern_word = '|'.join(target_action)
    
    df_temp[new_column] = (df_temp[column_target]
                         .str
                         .contains(pattern_word)
                         .astype(int)
                        )

    df_sort = df_temp.sort_values(by=[new_column], ascending=False)

    df_finish = df_sort.drop_duplicates(subset=drop_dup_column, keep='first')
    
    return df_finish

SyntaxError: non-default argument follows default argument (3721133969.py, line 10)

In [9]:
def device_os_pre(df: pd.DataFrame) -> pd.DataFrame:
    
    shame_df = df.shape[0]
    column_uniq = []
    
    for column in df.columns:
        if len(df[column].unique()) == shame_df:
            column_uniq.append(column)
            break
    
    df_combinations = df[
    ['device_category', 
     'device_brand', 
     'device_os']
    ].drop_duplicates().dropna()
    
    df_combinations.drop(
        df_combinations
        .loc[df_combinations['device_os'] == '(not set)']
        .index, 
        inplace=True
    )

    df = df.merge(df_combinations, 
                       on=['device_category', 'device_brand'], 
                       how='left', 
                       suffixes=('', '_y')
                      )

    df_clear = df.drop('device_os', axis=1).dropna()

    df_finish = df_clear[~df_clear[column_uniq].duplicated()]
    
    return df_finish

In [10]:
def organic_1_and_pay_0_traffic(df: pd.DataFrame) -> pd.DataFrame:
    
    df['organic_traff'] = np.where(df['utm_medium'].isin(organic_traffic), 1, 0)
    
    return df

### additional developments

In [1]:
###
##
#---> do делать функцию для генерации ключевого слова utm_keyword
#---> 
#---> poka dymay ydalit
##
###

# a, b = zip(*(data_sessions_pkl.utm_keyword.value_counts(normalize=True).head(3)).items())

# b = list(b)
# sum1 = 0.0
# for x in b:
#     sum1 += x
# sum2 = 1 - sum1
# valu = sum2 / 20
# valu1 = [valu * 17.5, valu * 1.5, valu * 1]
# finish = []
# chil = 0
# for x in range(3):
#     if chil == 3:
#         break
#     finish.append(b[chil] + valu1[x])
#     if chil < 3:
#         chil+=1
# print(finish, a)
# print(np.random.choice(a, p=finish))

# Preparatory work

## Downloads

In [22]:
data_hits_pkl = pd.read_pickle('../data/ga_hits.pkl')

In [12]:
data_sessions_pkl = pd.read_pickle('../data/ga_sessions.pkl')

## ga_hits_pkl

In [15]:
categorical_values(data_hits_pkl)

all line: 15726470


session_id        1734610
hit_date              226
hit_time           925888
hit_number            500
hit_type                1
hit_referer         37874
hit_page_path      342715
event_category         52
event_action          230
event_label         39826
event_value             1
dtype: int64

In [23]:
data_hits_pkl['hit_date'] = pd.to_datetime(data_hits_pkl['hit_date'])
data_hits_pkl['event_category'] = data_hits_pkl['event_category'].astype('category')

In [ ]:
data_hits_pkl.info()

## ga_sessions_pkl

In [13]:
categorical_values(data_sessions_pkl)

all line: 1860042


session_id                  1860042
client_id                   1391719
visit_date                      226
visit_time                    85318
visit_number                    537
utm_source                      294
utm_medium                       56
utm_campaign                    413
utm_adcontent                   287
utm_keyword                    1220
device_category                   3
device_os                        14
device_brand                    207
device_model                    105
device_screen_resolution       5039
device_browser                   57
geo_country                     166
geo_city                       2548
dtype: int64

In [14]:
data_sessions_pkl['visit_date'] = pd.to_datetime(data_sessions_pkl['visit_date'])
data_sessions_pkl[
[
    'device_category', 
    'device_os', 
    'utm_medium', 
    'device_brand', 
    'geo_country'
]
] = data_sessions_pkl[
[
    'device_category', 
    'device_os', 
    'utm_medium', 
    'device_brand', 
    'geo_country'
]
].astype('category')


In [10]:
data_sessions_pkl.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1860042 entries, 0 to 1860041
Data columns (total 18 columns):
 #   Column                    Dtype         
---  ------                    -----         
 0   session_id                object        
 1   client_id                 object        
 2   visit_date                datetime64[ns]
 3   visit_time                object        
 4   visit_number              int64         
 5   utm_source                object        
 6   utm_medium                category      
 7   utm_campaign              object        
 8   utm_adcontent             object        
 9   utm_keyword               object        
 10  device_category           category      
 11  device_os                 category      
 12  device_brand              category      
 13  device_model              object        
 14  device_screen_resolution  object        
 15  device_browser            object        
 16  geo_country               category      
 17  geo_city

# EXPLORATORY DATA ANALYSIS

## CLEANING

### ga_hits.pkl

In [ ]:
data_hits_pkl.info()

In [ ]:
data_hits_pkl = data_hits_pkl.drop(drop_columns_hit, axis=1)
data_hits_pkl.shape

In [51]:
df_hits = hit_time_median_category_event(data_hits_pkl)

In [52]:
df_hits.shape

(15726470, 9)

In [54]:
percent_null(df_hits)

Процент отсутствующих данных в df:


session_id         0.00
hit_date           0.00
hit_time           0.56
hit_number         0.00
hit_referer       39.90
hit_page_path      0.00
event_category     0.00
event_action       0.00
event_label       23.91
dtype: float64

In [66]:
df_hits = df_hits.dropna()
df_hits.shape

(15638025, 7)

In [67]:
df_hits = no_duplicated_target_and_null(df_hits, 
                                        target_action, 
                                        'target', 
                                        'event_action', 
                                        'session_id')
df_hits.shape

(1729595, 8)

In [69]:
print('Конечная структура df_hits:')
print(f'{rep, df_hits.info()}')

Конечная структура df_hits:
<class 'pandas.core.frame.DataFrame'>
Index: 1734610 entries, 10413010 to 5258378
Data columns (total 8 columns):
 #   Column          Dtype  
---  ------          -----  
 0   session_id      object 
 1   hit_date        object 
 2   hit_time        float64
 3   hit_number      int64  
 4   hit_page_path   object 
 5   event_category  object 
 6   event_action    object 
 7   target          int32  
dtypes: float64(1), int32(1), int64(1), object(5)
memory usage: 112.5+ MB
('request .info()', None)


### ga_sessions.pkl

In [15]:
percent_null(data_sessions_pkl)

values line: 1860042, columns: 18
Процент отсутствующих данных в df:


session_id                   0.00
client_id                    0.00
visit_date                   0.00
visit_time                   0.00
visit_number                 0.00
utm_source                   0.01
utm_medium                   0.00
utm_campaign                11.81
utm_adcontent               18.04
utm_keyword                 58.17
device_category              0.00
device_os                   57.53
device_brand                 6.38
device_model                99.12
device_screen_resolution     0.00
device_browser               0.00
geo_country                  0.00
geo_city                     0.00
dtype: float64

In [16]:
df_sessions = data_sessions_pkl.drop(drop_columns_sessions, axis=1)

In [17]:
df_sessions_clear = device_os_pre(df_sessions)
percent_null(df_sessions_clear)

values line: 1230781, columns: 16
Процент отсутствующих данных в df:


session_id                  0.0
client_id                   0.0
visit_date                  0.0
visit_time                  0.0
visit_number                0.0
utm_source                  0.0
utm_medium                  0.0
utm_campaign                0.0
utm_adcontent               0.0
device_category             0.0
device_brand                0.0
device_screen_resolution    0.0
device_browser              0.0
geo_country                 0.0
geo_city                    0.0
device_os_y                 0.0
dtype: float64

In [18]:
df_sessions_clear = organic_1_and_pay_0_traffic(df_sessions_clear)
df_sessions_clear.organic_traff.value_counts()

organic_traff
0    891687
1    339094
Name: count, dtype: int64

In [20]:
df_sessions_clear.isna().sum()

session_id                  0
client_id                   0
visit_date                  0
visit_time                  0
visit_number                0
utm_source                  0
utm_medium                  0
utm_campaign                0
utm_adcontent               0
device_category             0
device_brand                0
device_screen_resolution    0
device_browser              0
geo_country                 0
geo_city                    0
device_os_y                 0
organic_traff               0
dtype: int64

In [21]:
df_sessions_clear.to_pickle('test_df_session.pkl')